In [1]:
# Import necessary libraries
import os
import cv2
import torch
import logging
import subprocess
import pandas as pd
from pathlib import Path
logging.basicConfig(filename='process.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
import torch
print(torch.cuda.is_available())


True


In [ ]:
import os
import subprocess
from pathlib import Path
import logging

# Initialize logging
logging.basicConfig(
    filename='yolo_training.log', 
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def validate_directories(train_data_dir, train_labels_dir):
    """Check if training data and labels directories exist."""
    logging.info("Validating existence of training data directories...")
    if not train_data_dir.exists() or not train_labels_dir.exists():
        logging.error(f"Training data directories not found: {train_data_dir} or {train_labels_dir}")
        raise FileNotFoundError(f"Training data directories not found: {train_data_dir} or {train_labels_dir}")
    logging.info("Training data directories verified.")

def validate_data_yaml(data_yaml_path):
    """Check if the data.yaml file exists."""
    logging.info(f"Checking existence of data.yaml file at {data_yaml_path}...")
    if not Path(data_yaml_path).exists():
        logging.error(f"data.yaml file not found at {data_yaml_path}")
        raise FileNotFoundError(f"data.yaml file not found at {data_yaml_path}")
    logging.info(f"data.yaml file found at {data_yaml_path}")

def run_training(train_command):
    """Run the YOLOv5 training using subprocess."""
    try:
        result = subprocess.run(train_command, capture_output=True, text=True)
        logging.info(f"YOLOv5 training command executed: {train_command}")
    except Exception as e:
        logging.error(f"Exception occurred while executing YOLOv5 training: {e}")
        raise
    
    if result.returncode == 0:
        logging.info("YOLOv5 training completed successfully.")
        logging.info(f"Training output: {result.stdout}")
    else:
        logging.error(f"YOLOv5 training failed with error: {result.stderr}")
        raise SystemExit(f"YOLOv5 training failed with exit code {result.returncode}")

def find_latest_weights(weights_dir):
    """Find the latest experiment folder and return paths for best.pt and last.pt."""
    logging.info(f"Scanning for experiment folders in {weights_dir}...")
    exp_folders = [f for f in os.listdir(weights_dir) if 'exp' in f]
    
    if not exp_folders:
        logging.error(f"No experiment folders found in {weights_dir}.")
        raise FileNotFoundError(f"No experiment folders found in {weights_dir}")
    
    latest_exp_folder = sorted(exp_folders, key=lambda x: os.path.getmtime(os.path.join(weights_dir, x)))[-1]
    logging.info(f"Latest experiment folder identified: {latest_exp_folder}")
    
    best_weights_path = os.path.join(weights_dir, latest_exp_folder, 'weights', 'best.pt')
    last_weights_path = os.path.join(weights_dir, latest_exp_folder, 'weights', 'last.pt')
    
    return best_weights_path, last_weights_path, latest_exp_folder

def handle_weights(best_weights_path, last_weights_path, new_weights_path):
    """Handle renaming of weights files."""
    logging.info("Checking for best.pt or last.pt model weights...")
    
    if os.path.exists(best_weights_path):
        logging.info(f"Found best.pt at {best_weights_path}. Renaming to {new_weights_path}")
        if os.path.exists(new_weights_path):
            logging.info(f"Overwriting existing weights file at {new_weights_path}")
            os.remove(new_weights_path)
        os.rename(best_weights_path, new_weights_path)
        logging.info(f'Best weights saved to {new_weights_path}')
    elif os.path.exists(last_weights_path):
        logging.info(f"best.pt not found. Using last.pt from {last_weights_path} instead.")
        if os.path.exists(new_weights_path):
            logging.info(f"Overwriting existing weights file at {new_weights_path}")
            os.remove(new_weights_path)
        os.rename(last_weights_path, new_weights_path)
        logging.info(f'Last weights saved to {new_weights_path}')
    else:
        logging.error(f"Neither best.pt nor last.pt found.")
        raise FileNotFoundError(f"Neither best.pt nor last.pt found.")


if __name__ == "__main__":
    logging.info("YOLOv5 training script started.")
    
    # Define directories for training data (images and labels)
    train_data_dir = Path(r'../Data/medical_business_data/augmented_data/images')
    train_labels_dir = Path(r'../Data/medical_business_data/augmented_data/labels')
    
    # Validate directories
    validate_directories(train_data_dir, train_labels_dir)
    
    epochs = 30  
    batch_size = 4  
    img_size = 640 
    logging.info(f"Training parameters - Epochs: {epochs}, Batch Size: {batch_size}, Image Size: {img_size}")
    
    # Path to the data.yaml file (contains paths to the dataset and configuration)
    data_yaml_path = '../Data/medical_business_data/data.yaml'
    validate_data_yaml(data_yaml_path)
    
    logging.info("Starting YOLOv5 training...")
    train_command = [
        'python', 
        '../yolov5/train.py', 
        '--img', str(img_size), 
        '--batch', str(batch_size), 
        '--epochs', str(epochs), 
        '--data', data_yaml_path, 
        '--weights', 'yolov5s.pt', 
        '--cache',
        '--device', '0'
    ]
    
    # Run the training
    run_training(train_command)
    
    # Handle weights after training
    new_weights_path = 'labeledyolov5.pt'
    weights_dir = '../yolov5/runs/train'
    
    # Find and move weights
    best_weights_path, last_weights_path, latest_exp_folder = find_latest_weights(weights_dir)
    handle_weights(best_weights_path, last_weights_path, new_weights_path)
    
    logging.info("YOLOv5 training script completed.")
